In [18]:
import pandas as pd
from owlready2 import *
from rdflib import Graph
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.graph import Collection
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc.
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
from rdflib import Namespace, BNode
import rdflib

In [19]:
propertyy = pd.read_csv('10_Property_stolen_and_recovered.csv')
rape_victims = pd.read_csv('20_Victims_of_rape.csv')
Auto_theft = pd.read_csv('30_Auto_theft.csv')
murder_victims = pd.read_csv('32_Murder_victim_age_sexaa.csv')
firearms = pd.read_csv('34_Use_of_fire_arms_in_murder_cases.csv')
kidnapping = pd.read_csv('39_Specific_purpose_of_kidnapping_and_abduction.csv')
complaints = pd.read_csv('27_Nature_of_complaints_received_by_police.csv')
motives = pd.read_csv('19_Motive_or_cause_of_murder_and_culpable_homicide_not_amounting_to_murder.csv')
arrests = pd.read_csv('22_Persons_arrested_under_recidivism.csv')

complaints = complaints[['Area_Name','Year','PC1_Oral_Complaints','PC2_Written_Complaints']]
kidnapping = kidnapping[['Area_Name','Year','K_A_Grand_Total']]
motives = motives[['Area_Name','Year','CHNAMurder_Cause_Class_Conflict','CHNAMurder_Cause_For_Political_reason','Murder_Cause_Property_Dispute','Murder_Cause_Witchcraft']]
arrests = arrests[['Area_Name','Year','Offenders_Arrested']]

arrests

,Area_Name,Year,Offenders_Arrested
0,Uttar Pradesh,2001,314055
1,Maharashtra,2008,311598
2,Maharashtra,2010,305629
3,Madhya Pradesh,2010,343192
4,Uttar Pradesh,2010,292050
...,...,...,...
345,Lakshadweep,2007,26
346,Lakshadweep,2001,16
347,Lakshadweep,2010,15
348,Lakshadweep,2002,15


In [50]:

g = Graph()
g.parse('OntoCrime.owl')
myOntology =  Namespace("http://www.krrproject.com/CrimeOntology/")
g.bind("Crime", myOntology)
# crimy = URIRef('http://www.krrproject.com/CrimeOntology#')

In [51]:
# typesofcrime = [myOntology.violentcrime,myOntology.organizedcrime,myOntology.propertycrime,myOntology.whitecollarcrime]

# for i in typesofcrime:
#     g.add((myOntology.Crimes,myOntology.typesOfCrimes,i))


#addig years
dbp = Namespace('http://dbpedia.org/property/')
dbo = Namespace('http://dbpedia.org/ontology/')
dbr = Namespace('http://dbpedia.org/resource/') 

# populating Area
Area = URIRef(myOntology['Area'])
# populating propertyCrime
PR = URIRef(myOntology['PropertyCrime'])
g.add((PR,myOntology.hasPropertyCrime, myOntology.propertycrime))

for i in range(len(propertyy)):
    g.add((myOntology.propertycrime,myOntology.CrimeCommited,myOntology.propertycrime+str(i)))
    g.add((myOntology.propertycrime,RDFS.label,Literal("Property Crime",datatype=XSD.string)))
    
    # adding that property crime's date, year, sub group, cases of property stolen and value of property stolen
    g.add((myOntology.propertycrime+str(i),RDFS.label, Literal(propertyy['Sub_Group_Name'][i], datatype=XSD.string)))
    g.add((myOntology.propertycrime+str(i),myOntology.ValueOfProperty, Literal(propertyy['Value_of_Property_Stolen'][i], datatype=XSD.double)))
    g.add((myOntology.propertycrime+str(i),myOntology.NumberOfCases, Literal(propertyy['Cases_Property_Stolen'][i], datatype=XSD.integer)))
    g.add((myOntology.propertycrime+str(i),dbo.year, Literal(propertyy['Year'][i], datatype=XSD.gYear)))    
    g.add((myOntology.propertycrime+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string))) 
    g.add((myOntology.propertycrime+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string))) 
    
    

In [52]:
murder_victimsss = murder_victims.dropna(axis=0).reset_index()

In [53]:
# populating violentCrime (victims of murder by firearm)
g.add((myOntology.violentcrime,myOntology.hasViolentCrime, myOntology.murder))
g.add((myOntology.violentcrime,RDFS.label, Literal("Violent Crime",datatype=XSD.string)))

g.add((myOntology.murder,myOntology.hasVictims,myOntology.murderByFireArm))
for i in range(len(murder_victimsss)):
    g.add((myOntology.murderByFireArm,myOntology.CrimeCommited,myOntology.murderByFireArm+str(i)))
    g.add((myOntology.murderByFireArm+str(i),myOntology.NumberOfVictims,Literal(murder_victimsss['Victims_Total'][i], datatype=XSD.integer)))
    g.add((myOntology.murderByFireArm+str(i),dbo.state, Literal(murder_victimsss['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.murderByFireArm+str(i),dbo.year,Literal(murder_victimsss['Year'][i], datatype=XSD.gYear)))
    g.add((myOntology.murderByFireArm+str(i),myOntology.TypeOfWeapon,Literal(murder_victimsss['TypeOfFireArm'][i], datatype=XSD.string)))
    g.add((myOntology.murderByFireArm+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))  
    

# PUPOLATING RAPE CRIME
g.add((myOntology.violentcrime,myOntology.hasViolentCrime, myOntology.rape))
ranges = [
            myOntology.Victims_Upto_10_15_Yrs,
            myOntology.Victims_Upto_10_Yrs_y,
            myOntology.Victims_Upto_15_18_Yrs,
            myOntology.Victims_Upto_18_30_Yrs,
            myOntology.Victims_Upto_30_50_Yrs
         ]
g.add((myOntology.rape,myOntology.hasAgeRanges,ranges[0]))
g.add((myOntology.rape,myOntology.hasAgeRanges,ranges[1]))
g.add((myOntology.rape,myOntology.hasAgeRanges,ranges[2]))
g.add((myOntology.rape,myOntology.hasAgeRanges,ranges[3]))
g.add((myOntology.rape,myOntology.hasAgeRanges,ranges[4]))


for i in range(len(murder_victimsss)):
    g.add((ranges[0],myOntology.hasVictims,ranges[0]+str(i)))
    g.add((ranges[0]+str(i),myOntology.NumberOfVictims,Literal(murder_victimsss['Victims_Upto_10_15_Yrs'][i], datatype=XSD.integer)))
    g.add((ranges[0]+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((ranges[0]+str(i),dbo.year,Literal(murder_victimsss['Year'][i], datatype=XSD.gYear)))
    g.add((ranges[0]+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    

    g.add((ranges[1],myOntology.hasVictims,ranges[1]+str(i)))
    g.add((ranges[1]+str(i),myOntology.NumberOfVictims,Literal(murder_victimsss['Victims_Upto_10_Yrs'][i], datatype=XSD.integer)))
    g.add((ranges[1]+str(i),dbo.state, Literal(murder_victimsss['Area_Name'][i], datatype=XSD.string)))
    g.add((ranges[1]+str(i),dbo.year,Literal(murder_victimsss['Year'][i], datatype=XSD.gYear)))
    g.add((ranges[1]+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    

    g.add((ranges[2],myOntology.hasVictims,ranges[2]+str(i)))
    g.add((ranges[2]+str(i),myOntology.NumberOfVictims,Literal(murder_victimsss['Victims_Upto_15_18_Yrs'][i], datatype=XSD.integer)))
    g.add((ranges[2]+str(i),dbo.state, Literal(murder_victimsss['Area_Name'][i], datatype=XSD.string)))
    g.add((ranges[2]+str(i),dbo.year,Literal(murder_victimsss['Year'][i], datatype=XSD.gYear)))
    g.add((ranges[2]+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    
    
    g.add((ranges[3],myOntology.hasVictims,ranges[3]+str(i)))
    g.add((ranges[3]+str(i),myOntology.NumberOfVictims,Literal(murder_victimsss['Victims_Upto_18_30_Yrs'][i], datatype=XSD.integer)))
    g.add((ranges[3]+str(i),dbo.state, Literal(murder_victimsss['Area_Name'][i], datatype=XSD.string)))
    g.add((ranges[3]+str(i),dbo.year,Literal(murder_victimsss['Year'][i], datatype=XSD.gYear)))
    g.add((ranges[3]+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))  
    
    
    g.add((ranges[4],myOntology.hasVictims,ranges[4]+str(i)))
    g.add((ranges[4]+str(i),myOntology.NumberOfVictims,Literal(murder_victimsss['Victims_Upto_30_50_Yrs'][i], datatype=XSD.integer)))
    g.add((ranges[4]+str(i),dbo.state, Literal(murder_victimsss['Area_Name'][i], datatype=XSD.string)))
    g.add((ranges[4]+str(i),dbo.year,Literal(murder_victimsss['Year'][i], datatype=XSD.gYear)))
    g.add((ranges[4]+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    
    


In [54]:
# kidnapping = kidnapping.dropna(axis = 0).reset_index()
# kidnapping.isnull().sum()

In [55]:
#populating organized crime (kidnapping) 
g.add((myOntology.organizedcrime,myOntology.hasOrganizedCrime, myOntology.kidnapping))
g.add((myOntology.organizedcrime,RDFS.label, Literal("Organized Crime",datatype=XSD.string)))

for i in range(len(kidnapping)):
    g.add((myOntology.kidnapping,myOntology.hasVictims,myOntology.kidnapping+str(i)))    
    g.add((myOntology.kidnapping+str(i),myOntology.NumberOfVictims,Literal(kidnapping['K_A_Grand_Total'][i], datatype=XSD.integer)))
    g.add((myOntology.kidnapping+str(i),dbo.state, Literal(kidnapping['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.kidnapping+str(i),dbo.year,Literal(kidnapping['Year'][i], datatype=XSD.gYear)))  
    g.add((myOntology.kidnapping+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))  
    g.add((myOntology.kidnapping+str(i),RDFS.label,Literal("Kidnapping",datatype=XSD.string))) 
    
    
    

In [56]:
complaints = complaints.dropna(axis=0).reset_index()
complaints.isnull().sum()

index                     0
Area_Name                 0
Year                      0
PC1_Oral_Complaints       0
PC2_Written_Complaints    0
dtype: int64

In [57]:
# populatings complaints recieved by police 
g.add((myOntology.CrimeDepartment,FOAF.Organization, myOntology.policeDepartment))
g.add((myOntology.policeDepartment,myOntology.hasComplaints, myOntology.OralComplaints))
g.add((myOntology.policeDepartment,myOntology.hasComplaints, myOntology.WrittenComplaints))

for i in range(0,len(complaints)):
    
    # oral complaints, their area and thier year 
    g.add((myOntology.OralComplaints,myOntology.hasComplaints,myOntology.OralComplaints+str(i)))
    g.add((myOntology.OralComplaints+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.OralComplaints+str(i),dbo.year,Literal(complaints['Year'][i], datatype=XSD.gYear)))   
    g.add((myOntology.OralComplaints+str(i),myOntology.NumberofComplaints,Literal(complaints['PC1_Oral_Complaints'][i],datatype=XSD.integer )))
    g.add((myOntology.OralComplaints+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    g.add((myOntology.OralComplaints+str(i),RDFS.label,Literal("Oral Complaints",datatype=XSD.string))) 
    
    
    
    # written complaints, their area and thier year 
    g.add((myOntology.WrittenComplaints,myOntology.hasComplaints,myOntology.WrittenComplaints+str(i)))
    g.add((myOntology.WrittenComplaints+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.WrittenComplaints+str(i),dbo.year,Literal(complaints['Year'][i], datatype=XSD.gYear)))   
    g.add((myOntology.WrittenComplaints+str(i),myOntology.NumberofComplaints,Literal(complaints['PC2_Written_Complaints'][i],datatype=XSD.integer )))
    g.add((myOntology.WrittenComplaints+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    g.add((myOntology.WrittenComplaints+str(i),RDFS.label,Literal("Written Complaints",datatype=XSD.string))) 
    
    
    
# persons arrested under recividism (former prisoner committing the same crime again)
g.add((myOntology.PoliceDep,myOntology.hasArrests, myOntology.offender))
for i in range(len(arrests)):
    g.add((myOntology.offender,myOntology.hasArrests, myOntology.offender+str(i)))
    g.add((myOntology.offender+str(i),myOntology.NumberOfOffendersArrested,Literal(arrests['Offenders_Arrested'][i],datatype=XSD.integer)))
    g.add((myOntology.offender+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.offender+str(i),dbo.year,Literal(arrests['Year'][i], datatype=XSD.gYear)))
    g.add((myOntology.offender+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    


In [58]:
arrests

,Area_Name,Year,Offenders_Arrested
0,Uttar Pradesh,2001,314055
1,Maharashtra,2008,311598
2,Maharashtra,2010,305629
3,Madhya Pradesh,2010,343192
4,Uttar Pradesh,2010,292050
...,...,...,...
345,Lakshadweep,2007,26
346,Lakshadweep,2001,16
347,Lakshadweep,2010,15
348,Lakshadweep,2002,15


In [59]:
# populating motives
g.add((myOntology.murder,myOntology.hasMotive, myOntology.classConflict))
g.add((myOntology.murder,myOntology.hasMotive, myOntology.propertyDispute))
g.add((myOntology.murder,myOntology.hasMotive, myOntology.politicalMotive))
g.add((myOntology.murder,myOntology.hasMotive, myOntology.witchCraft))

for i in range(len(motives)):
    # motive of muder witch craft
    g.add((myOntology.witchCraft,myOntology.hasMotive,myOntology.witchCraft+str(i)))
    g.add((myOntology.witchCraft+str(i),myOntology.NumberOfMurders,Literal(motives['Murder_Cause_Witchcraft'][i],datatype=XSD.integer )))
    g.add((myOntology.witchCraft+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.witchCraft+str(i),dbo.year,Literal(motives['Year'][i], datatype=XSD.gYear))) 
    g.add((myOntology.witchCraft+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    g.add((myOntology.witchCraft+str(i),RDFS.label,Literal("Murder_Cause_Witchcraft",datatype=XSD.string))) 
    
    
    # motive of muder political motive
    g.add((myOntology.politicalMotive,myOntology.hasMotive,myOntology.politicalMotive+str(i)))
    g.add((myOntology.politicalMotive+str(i),myOntology.NumberOfMurders,Literal(motives['CHNAMurder_Cause_For_Political_reason'][i],datatype=XSD.integer )))
    g.add((myOntology.politicalMotive+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.politicalMotive+str(i),dbo.year,Literal(motives['Year'][i], datatype=XSD.gYear))) 
    g.add((myOntology.politicalMotive+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))  
    g.add((myOntology.politicalMotive+str(i),RDFS.label,Literal("CHNAMurder_Cause_For_Political_reason",datatype=XSD.string))) 
    
    
     # motive of muder propertyDispute motive
    g.add((myOntology.propertyDispute,myOntology.hasMotive,myOntology.propertyDispute+str(i)))
    g.add((myOntology.propertyDispute+str(i),myOntology.NumberOfMurders,Literal(motives['Murder_Cause_Property_Dispute'][i],datatype=XSD.integer )))
    g.add((myOntology.propertyDispute+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.propertyDispute+str(i),dbo.year,Literal(motives['Year'][i], datatype=XSD.gYear)))
    g.add((myOntology.propertyDispute+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    g.add((myOntology.propertyDispute+str(i),RDFS.label,Literal("Murder_Cause_Property_Dispute",datatype=XSD.string))) 
    
    
     # motive of muder class conflict motive
    g.add((myOntology.classConflict,myOntology.hasMotive,myOntology.classConflict+str(i)))
    g.add((myOntology.classConflict+str(i),myOntology.NumberOfMurders,Literal(motives['CHNAMurder_Cause_Class_Conflict'][i],datatype=XSD.integer )))
    g.add((myOntology.classConflict+str(i),dbo.state, Literal(propertyy['Area_Name'][i], datatype=XSD.string)))
    g.add((myOntology.classConflict+str(i),dbo.year,Literal(motives['Year'][i], datatype=XSD.gYear)))  
    g.add((myOntology.classConflict+str(i),dbo.country,dbr.India))
    g.add((dbr.India,RDFS.label,Literal("India",datatype=XSD.string)))   
    g.add((myOntology.classConflict+str(i),RDFS.label,Literal("CHNAMurder_Cause_Class_Conflict",datatype=XSD.string))) 
    
    


    

In [60]:
g.add((myOntology.victimlesscrime,RDFS.label, Literal("Organized Crime",datatype=XSD.string)))
g.add((myOntology.whitecollarcrime,RDFS.label, Literal("Organized Crime",datatype=XSD.string)))

<Graph identifier=Nee991450169a47808b3cf8269431d711 (<class 'rdflib.graph.Graph'>)>

In [61]:
g.serialize(destination='CrimeOntology_RDF_Graph2.ttl')  

<Graph identifier=Nee991450169a47808b3cf8269431d711 (<class 'rdflib.graph.Graph'>)>

In [19]:
print(g.serialize(format='turtle')) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [52]:
propertyy

,Area_Name,Year,Group_Name,Sub_Group_Name,Cases_Property_Recovered,Cases_Property_Stolen,Value_of_Property_Recovered,Value_of_Property_Stolen
0,Andaman & Nicobar Islands,2001,Burglary - Property,3. Burglary,27,64,755858,1321961
1,Andhra Pradesh,2001,Burglary - Property,3. Burglary,3321,7134,51483437,147019348
2,Arunachal Pradesh,2001,Burglary - Property,3. Burglary,66,248,825115,4931904
3,Assam,2001,Burglary - Property,3. Burglary,539,2423,3722850,21466955
4,Bihar,2001,Burglary - Property,3. Burglary,367,3231,2327135,17023937
...,...,...,...,...,...,...,...,...
2444,Tamil Nadu,2010,Total Property,7. Total Property Stolen & Recovered,16125,21509,660311804,1317919190
2445,Tripura,2010,Total Property,7. Total Property Stolen & Recovered,192,879,5666102,33032746
2446,Uttar Pradesh,2010,Total Property,7. Total Property Stolen & Recovered,9130,35068,577591772,1442670414
2447,Uttarakhand,2010,Total Property,7. Total Property Stolen & Recovered,964,2234,47135685,123398840
